In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import cv2
import time
import pickle
import glob
import os
import json
import tensorflow as tf
from tensorflow.keras.models import load_model
from skimage import color
from skimage.feature import hog
from scipy.ndimage.measurements import label
import matplotlib.pyplot as plt

In [2]:
def preprocessing_input_image(img, img_size):
    resized_image = np.resize(img, img_size)
    grayscale_image = color.rgb2gray(resized_image)
    features = hog(grayscale_image, 
                   orientations=30, 
                   pixels_per_cell=(16, 16), 
                   cells_per_block=(2, 2), 
                   block_norm='L2-Hys', 
                   visualize=False, 
                   transform_sqrt=False, 
                   feature_vector=True, 
                   multichannel=None)
    return features

In [3]:
def sliding_window(image, xy_window, xy_overlap, image_size):
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    nx_windows = np.int(image_size[0]/nx_pix_per_step) - 1
    ny_windows = np.int(image_size[1]/ny_pix_per_step) - 1
    window_list = []
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            startx = xs*nx_pix_per_step
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step
            endy = starty + xy_window[1]
            window_list.append(((startx, starty), (endx, endy)))
    return window_list

In [4]:
def search_windows(img, windows, scaler, classifier):
    on_windows = []
    for window in windows:
        feature_image = preprocessing_input_image(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (128, 128,3))
        feature_image = scaler.transform(feature_image.reshape(1,-1))
        prediction = classifier.predict(feature_image)
        if max(prediction)>0.8:
            if classifier.predict_classes(feature_image) is not 0:
                on_windows.append(window)
    return on_windows

In [5]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    imcopy = np.copy(img)
    for bbox in bboxes:
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    return imcopy

In [6]:
def add_heat(heatmap, bbox_list):
    for box in bbox_list:
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap

In [7]:
def apply_threshold(heatmap, threshold):
    heatmap[heatmap <= threshold] = 0
    return heatmap

In [8]:
def draw_labeled_bboxes(img, labels):
    for product_number in range(1, labels[1]+1):
        nonzero = (labels[0] == product_number).nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
#         bbox_w = (bbox[1][0] - bbox[0][0])
#         bbox_h = (bbox[1][1] - bbox[0][1])
#         # Filter final detections for aspect ratios, e.g. thin vertical box is likely not a car
#         aspect_ratio = bbox_w / bbox_h  # width / height
#         #print('ar: %s' % (aspect_ratio,))

#         # Also if small box "close" to the car (i.e. bounding box y location is high),
#         # then probaby not a car
#         bbox_area = bbox_w * bbox_h
#         # Combine above filters with minimum bbox area filter
# #         if aspect_ratio > 0.5 and aspect_ratio < max_ar and not small_box_close and bbox_area > min_bbox_area:
#             # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)

    return img

In [9]:
def object_detection_pipeline():
    with open('config.json') as json_file:
        data = json.load(json_file)
    train_scaler = pickle.load( open( "scalers//hog_product_scaler.pkl", "rb" ) )
    product_classifier = load_model("models//hog_product_model.h5")
    count_dict = {}
    for image_file in glob.glob(data["object_detector_directory"]["test_object_dir"]+"*.JPG"):
        image = cv2.imread(image_file)
        resized = cv2.resize(image, (350, 350), interpolation=cv2.INTER_CUBIC)
        draw_image = np.copy(resized)
        windows = sliding_window(resized, (30, 55), (0.7, 0.7), (350, 350))
        hot_windows = search_windows(resized, windows, train_scaler, product_classifier)
        window_img = draw_boxes(resized, hot_windows , color=(0, 0, 255), thick=6)
#         plt.imshow(window_img)
#         plt.show()
        heatmap = np.zeros((350, 350))  # NOTE: Image dimensions hard-coded
        heatmap = add_heat(heatmap, hot_windows)
        heatmap = apply_threshold(heatmap, 2)
        labels = label(heatmap)
#         print(labels[1], 'products found')
#         plt.imshow(labels[0], cmap='gray')
#         plt.show()
            # Draw final bounding boxes
        draw_img = draw_labeled_bboxes(np.copy(resized), labels)
        count_dict[os.path.basename(image_file)] = labels[1]
    out_file = open("image2products.json", "w") 
    json.dump(count_dict, out_file, indent = 6) 
    out_file.close() 

In [10]:
object_detection_pipeline()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
